In [2]:
import numpy as np
import pandas as pd
import dashboard
import utils

In [3]:
def parse_list_col(x: str) -> list[str]:
    x = x.replace('{', '[').replace('}', ']')
    return eval(x)

test_data = "{'sweet', 'creamy', 'caramel', 'lactonic', 'brown'}"
parse_list_col(test_data)


['sweet', 'creamy', 'caramel', 'lactonic', 'brown']

## Read in the molecules data file

In [3]:
molecules = pd.read_csv("data/flavordb/molecules.csv", index_col=0)
#convert molecules string ("{'sweet', 'creamy', 'caramel', 'lactonic', 'brown'}") to list
molecules['flavor profile'] = molecules['flavor profile'].apply(parse_list_col)
print(molecules.head())


   pubchem id                  common name  \
0           4           1-Aminopropan-2-ol   
1          49  3-Methyl-2-oxobutanoic acid   
2          58           2-oxobutanoic acid   
3          70   4-Methyl-2-oxovaleric acid   
4          72    3,4-Dihydroxybenzoic Acid   

                              flavor profile  
0                                    [fishy]  
1                                   [fruity]  
2  [sweet, creamy, caramel, lactonic, brown]  
3                                   [fruity]  
4                 [mild, balsamic, phenolic]  


## Read the flavordb.csv file


In [4]:
flavor_molecules = pd.read_csv("data/flavordb/flavordb.csv", index_col=0)
#convert the two list-like columns to list "{27457, 7976, 31252, 26808, 22201, 26331}"
flavor_molecules['synonyms'] = flavor_molecules['synonyms'].apply(parse_list_col)
flavor_molecules['molecules'] = flavor_molecules['molecules'].apply(parse_list_col)

print(flavor_molecules.head())
#print(flavor_molecules.describe())

   entity id            alias                   synonyms scientific name  \
0          1  bakery products          [bakery products]        poacceae   
1          2            bread                    [bread]        poacceae   
2          3        rye bread                [rye bread]             rye   
3          4    wheaten bread  [soda scones, soda farls]           wheat   
4          5      white bread              [white bread]           wheat   

  category                                          molecules  
0   bakery          [27457, 7976, 31252, 26808, 22201, 26331]  
1   bakery  [1031, 1032, 644104, 527, 8723, 31260, 15394, ...  
2   bakery  [644104, 7824, 643731, 8468, 1049, 5372954, 80...  
3   bakery  [6915, 5365891, 12170, 8082, 31251, 7958, 1049...  
4   bakery  [7361, 994, 10883, 7362, 11173, 5365891, 11559...  


### New way to load all data

In [4]:
config = dashboard.read_config("config.ini")
datastore = dashboard.read_all_data(config)

### Get info on a selected ingredient

In [5]:
selected_ingredient = 'japanese pumpkin'
#datastore.ingredients['alias'] == selected_ingredient
ingredient = datastore.ingredients[datastore.ingredients['alias'] == selected_ingredient]
print(ingredient)

     entity id             alias                     synonyms scientific name  \
932        972  japanese pumpkin  [kabocha, japanese pumpkin]   winter squash   

    category                                          molecules  
932    fruit  [644104, 527, 8723, 31260, 15394, 6184, 439341...  


In [6]:
ingredient_flavor_molecules = ingredient['molecules'].iloc[0]
ingredient_flavor_molecules

for i, id in enumerate(ingredient_flavor_molecules):
    # fetch flavor molecules
    fm = datastore.flavor_molecules[datastore.flavor_molecules['pubchem id'] == id]
    print(f'''count={i}, id={id}, name={fm.iloc[0, 1]}, flavor profile={fm.iloc[0, 2]}''')



count=0, id=644104, name=CID 644104, flavor profile=['very mild', 'grassy']
count=1, id=527, name=Propionaldehyde, flavor profile=['nutty', 'cocoa', 'pungent', 'whiskey', 'earthy', 'solvent', 'wine', 'alcohol']
count=2, id=8723, name=2-Methyl-1-Butanol, flavor profile=['onion', 'fruity', 'malt', 'wine', 'roasted']
count=3, id=31260, name=3-Methyl-1-butanol, flavor profile=['alcoholic', 'burnt', 'whiskey', 'fruity', 'malt', 'fusel', 'banana', 'oil']
count=4, id=15394, name=Ethyl Vinyl Ketone, flavor profile=['mustard', 'garlic', 'onion', 'pungent', 'fish', 'peppery']
count=5, id=6184, name=Hexanal, flavor profile=['green', 'fresh', 'leafy', 'aldehydic', 'fat', 'fruity', 'grass', 'sweaty', 'tallow', 'fatty']
count=6, id=439341, name=alpha-Maltose, flavor profile=['sweet', 'odorless']
count=7, id=5365811, name=4-Hexen-3-One, flavor profile=['tropical', 'green', 'pungent', 'metallic', 'spicy', 'ethereal']
count=8, id=6202, name=Thiamine Hydrochloride, flavor profile=['mild', 'sour', 'bitte

In [7]:
id = 644104
datastore.flavor_molecules['pubchem id'] == id
fm = datastore.flavor_molecules[datastore.flavor_molecules['pubchem id'] == id]
fm

,pubchem id,common name,flavor profile
1397,644104,CID 644104,"[very mild, grassy]"
